In [19]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

In [20]:
df = pd.read_csv('train.csv')

In [5]:
df.shape

(404290, 6)

In [36]:
new_df = df.sample(30000,random_state=2)

In [37]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
237389,237389,348542,348543,What are some ways São Tomé & Príncipe can dev...,What are some ways Australia can develop its e...,0
238282,238282,349611,349612,What are some good colleges for pursuing M.Tec...,What are the best colleges for pursuing an M.T...,0
318966,318966,333883,19554,What are some interview questions being asked ...,What are the questions in the interview for a ...,1
372195,372195,239868,102325,What are the best additional courses for worki...,What is the best programs for mechanical engin...,1
377861,377861,509201,509202,I've been eating white rice with tuna and 1 av...,Tuna is one of the most convenient protein sou...,0


In [38]:
new_df.shape

(30000, 6)

In [35]:
def preprocessing(q):
    q = str(q).lower().strip()

    # Replace certain special characters with their string equivalents
    q = q.replace("%", 'percent')
    q = q.replace("$", 'dollor')
    q = q.replace("₹", 'rupee')
    q = q.replace("€", 'euro')
    q = q.replace("@", 'at')

    # the pattern [math] appearns around 900 times in the whole dataset
    q = q.replace('[math]', '')

    # Decontracting words
    # https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions
    # https://stackoverflow.com/a/19794953
    contractions = { 
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

    q_decontracted = []

    for word in q.split():
        if word in contractions:
            word = contractions[word]
        q_decontracted.append(word)

    q = ' '.join(q_decontracted)

    q = q.replace("'ve", " have")
    q = q.replace("'re", " are")
    q = q.replace("n't", " not")
    q = q.replace("'ll", " will")


    # remove HTML

    q = BeautifulSoup(q)
    q = q.get_text()

    # remove punctuation
    pattern = re.compile('\W')
    q = re.sub(pattern, ' ', q).strip()

    return q
    

In [31]:
preprocessing("I've already! wasn't <b>done</b>?")

'i have already  was not done'

In [40]:
new_df['question1'] = df['question1'].apply(preprocessing)
new_df['question2'] = df['question2'].apply(preprocessing)

C:\Users\write\AppData\Local\Temp\ipykernel_10068\1043997485.py:154: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  q = BeautifulSoup(q)


In [41]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
237389,237389,348542,348543,what are some ways são tomé príncipe can dev...,what are some ways australia can develop its e...,0
238282,238282,349611,349612,what are some good colleges for pursuing m tec...,what are the best colleges for pursuing an m t...,0
318966,318966,333883,19554,what are some interview questions being asked ...,what are the questions in the interview for a ...,1
372195,372195,239868,102325,what are the best additional courses for worki...,what is the best programs for mechanical engin...,1
377861,377861,509201,509202,i have been eating white rice with tuna and 1 ...,tuna is one of the most convenient protein sou...,0


In [42]:
new_df['q1_len'] = new_df['question1'].str.len()
new_df['q2_len'] = new_df['question2'].str.len()

In [43]:
new_df['q1_num_word'] = new_df['question1'].apply(lambda x: len(x.split()))
new_df['q2_num_word'] = new_df['question2'].apply(lambda x: len(x.split()))

In [44]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_word,q2_num_word
237389,237389,348542,348543,what are some ways são tomé príncipe can dev...,what are some ways australia can develop its e...,0,62,52,11,9
238282,238282,349611,349612,what are some good colleges for pursuing m tec...,what are the best colleges for pursuing an m t...,0,73,74,12,14
318966,318966,333883,19554,what are some interview questions being asked ...,what are the questions in the interview for a ...,1,79,75,13,13
372195,372195,239868,102325,what are the best additional courses for worki...,what is the best programs for mechanical engin...,1,69,53,10,8
377861,377861,509201,509202,i have been eating white rice with tuna and 1 ...,tuna is one of the most convenient protein sou...,0,149,143,31,31


In [48]:
def common_words(row):
    c1 = set(map(lambda x: x.lower().strip(), row['question1'].split()))
    c2 = set(map(lambda x: x.lower().strip(), row['question2'].split()))

    return len(c1 & c2)

In [49]:
new_df['word_common'] = new_df.apply(common_words, axis=1)
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_word,q2_num_word,word_common
237389,237389,348542,348543,what are some ways são tomé príncipe can dev...,what are some ways australia can develop its e...,0,62,52,11,9,8
238282,238282,349611,349612,what are some good colleges for pursuing m tec...,what are the best colleges for pursuing an m t...,0,73,74,12,14,9
318966,318966,333883,19554,what are some interview questions being asked ...,what are the questions in the interview for a ...,1,79,75,13,13,6
372195,372195,239868,102325,what are the best additional courses for worki...,what is the best programs for mechanical engin...,1,69,53,10,8,5
377861,377861,509201,509202,i have been eating white rice with tuna and 1 ...,tuna is one of the most convenient protein sou...,0,149,143,31,31,4


In [87]:
def total_words(row):
    t1 = set(map(lambda x: x.lower().strip(), row['question1'].split()))
    t2 = set(map(lambda x: x.lower().strip(), row['question2'].split()))
    return len(t1) + len(t2)

In [88]:
new_df['word_total'] = new_df.apply(total_words, axis=1)

In [90]:
new_df['word_share'] = round(new_df['word_common'] / new_df['word_total'], 2)
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_word,q2_num_word,word_common,word_total,word_share
237389,237389,348542,348543,what are some ways são tomé príncipe can dev...,what are some ways australia can develop its e...,0,62,52,11,9,8,20,0.40
238282,238282,349611,349612,what are some good colleges for pursuing m tec...,what are the best colleges for pursuing an m t...,0,73,74,12,14,9,25,0.36
318966,318966,333883,19554,what are some interview questions being asked ...,what are the questions in the interview for a ...,1,79,75,13,13,6,25,0.24
372195,372195,239868,102325,what are the best additional courses for worki...,what is the best programs for mechanical engin...,1,69,53,10,8,5,18,0.28
377861,377861,509201,509202,i have been eating white rice with tuna and 1 ...,tuna is one of the most convenient protein sou...,0,149,143,31,31,4,52,0.08
